In [1]:
import os
root_dir_name = 'AmazonProductSearch'
curr_dir = os.getcwd()
root_dir = curr_dir[:curr_dir.index(root_dir_name) + len(root_dir_name)]
dataset_dir = os.path.join(root_dir , 'Dataset')
ml_dataset_dir = os.path.join(dataset_dir ,'MultiLingual')


In [4]:
import pandas as pd 
df_main = pd.read_parquet(os.path.join(ml_dataset_dir , 'finalized_dataset_forvectorization.parquet'))
df_queries = pd.read_parquet(os.path.join(dataset_dir , 'shopping_queries_dataset_examples.parquet'))
df_queries

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train
...,...,...,...,...,...,...,...,...,...
2621283,2621283,�����j�[�h�p�[�x abrasus,130651,B0063ASUY4,jp,E,0,1,test
2621284,2621284,�����j�[�h�p�[�x abrasus,130651,B0062EZYIG,jp,E,0,1,test
2621285,2621285,�����j�[�h�p�[�x abrasus,130651,B07H8MWBZN,jp,S,0,1,test
2621286,2621286,�����j�[�h�p�[�x abrasus,130651,B00IZH4T9S,jp,E,0,1,test


In [14]:
df_queries_h = df_queries[df_queries['small_version'] == 1]
df_queries_e = df_queries[df_queries['small_version'] == 0]

In [15]:
len(df_queries_e['query_id'].unique()), len(df_queries_h['query_id'].unique())

(82360, 48300)

In [19]:
df_queries_h['product_id'] = df_queries_h['product_locale'] + df_queries_h['product_id']
df_queries_e['product_id'] = df_queries_e['product_locale'] + df_queries_e['product_id']

C:\Users\prade\AppData\Local\Temp\ipykernel_24288\1586428510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries_h['product_id'] = df_queries_h['product_locale'] + df_queries_h['product_id']
C:\Users\prade\AppData\Local\Temp\ipykernel_24288\1586428510.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_queries_e['product_id'] = df_queries_e['product_locale'] + df_queries_e['product_id']


In [21]:
len(df_queries_e['query_id'].unique()), len(df_queries_h['query_id'].unique())

(82360, 48300)

In [22]:
trained_product_ids = df_main['product_id']

planning to consider a final of 10k easy , 5k hard - so give it around 33% of selection process

In [27]:
(df_queries_e[(df_queries_e['product_id'].isin(trained_product_ids)) & (df_queries_e['esci_label'].isin(['E','S']))]['query_id'].value_counts() > 4).sum() , \
 (df_queries_h[(df_queries_h['product_id'].isin(trained_product_ids)) & (df_queries_h['esci_label'].isin(['E','S']))]['query_id'].value_counts() > 4).sum()

(29787, 15203)

Stargies query selection process as we have removed a lot of products in the data preprocessing step

In [28]:
easy_sampled_query = (df_queries_e[(df_queries_e['product_id'].isin(trained_product_ids)) & (df_queries_e['esci_label'].isin(['E','S']))]['query_id'].value_counts() > 4)
easy_sampled_query_index = easy_sampled_query.index[easy_sampled_query.values]
hard_sampled_query = (df_queries_h[(df_queries_h['product_id'].isin(trained_product_ids)) & (df_queries_h['esci_label'].isin(['E','S']))]['query_id'].value_counts() > 4)
hard_sampled_query_index = hard_sampled_query.index[hard_sampled_query.values]

In [29]:
len(easy_sampled_query_index) , len(hard_sampled_query_index)

(29787, 15203)

In [30]:
df_queries_e = df_queries_e[df_queries_e['query_id'].isin(easy_sampled_query_index)]
df_queries_h = df_queries_h[df_queries_h['query_id'].isin(hard_sampled_query_index)]

In [31]:
# randomly selection of 10 K queries from each easy and hard a with random_state = 42
import random
random.seed(42)
easy_search_queries = random.sample(list(df_queries_e['query_id'].unique()) , 10000)
hard_search_queries = random.sample(list(df_queries_h['query_id'].unique()) , 5000)
easy_search_queries.sort()
hard_search_queries.sort()

In [32]:
len(easy_search_queries) , len(hard_search_queries)

(10000, 5000)

In [33]:
df_queries_e = df_queries_e[df_queries_e['query_id'].isin(easy_search_queries)]
df_queries_h = df_queries_h[df_queries_h['query_id'].isin(hard_search_queries)]

In [34]:
df_queries_e

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
2019,2019,$10 blanket,65,usB07V3MBK82,us,E,0,1,train
2020,2020,$10 blanket,65,usB08BJG8S76,us,E,0,1,train
2021,2021,$10 blanket,65,usB0871Y1FPD,us,E,0,1,train
2022,2022,$10 blanket,65,usB0837GR8RT,us,E,0,1,train
2023,2023,$10 blanket,65,usB0834D8MJ3,us,E,0,1,train
...,...,...,...,...,...,...,...,...,...
2620494,2620494,ｓｐｏ２,130615,jpB01LEA4R26,jp,E,0,1,train
2620495,2620495,ｓｐｏ２,130615,jpB01LEA4QYU,jp,E,0,1,train
2620496,2620496,ｓｐｏ２,130615,jpB005FDPBVK,jp,E,0,1,train
2620497,2620497,ｓｐｏ２,130615,jpB0057M8HP6,jp,E,0,1,train


In [35]:
len(df_queries_e['query_id'].unique()) , len(df_queries_h['query_id'].unique())

(10000, 5000)

In [36]:
df_queries_h

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
70,70,!solid camiseta sin manga,3,esB08TQH4RBB,es,E,1,1,train
71,71,!solid camiseta sin manga,3,esB08H2H18DN,es,E,1,1,train
72,72,!solid camiseta sin manga,3,esB08HJQJT2T,es,E,1,1,train
73,73,!solid camiseta sin manga,3,esB08LP68XTS,es,S,1,1,train
74,74,!solid camiseta sin manga,3,esB08NDS1HS2,es,E,1,1,train
...,...,...,...,...,...,...,...,...,...
2620981,2620981,ｻｳﾅﾏｯﾄ,130636,jpB07DPKTY7Y,jp,E,1,1,train
2620982,2620982,ｻｳﾅﾏｯﾄ,130636,jpB08DVCB8PM,jp,E,1,1,train
2620983,2620983,ｻｳﾅﾏｯﾄ,130636,jpB07BYJ5R9P,jp,S,1,1,train
2620984,2620984,ｻｳﾅﾏｯﾄ,130636,jpB07BC9VDNJ,jp,E,1,1,train


In [37]:
df_queries_e.to_parquet(os.path.join(ml_dataset_dir , 'easy_search_query_dataset.parquet'))
df_queries_h.to_parquet(os.path.join(ml_dataset_dir , 'hard_search_query_dataset.parquet'))


In [38]:
len(df_queries_e['query_id'].unique()) , len(df_queries_h['query_id'].unique())

(10000, 5000)